In [ ]:
import numpy as np
import MDAnalysis as mda
import pandas as pd
import nglview as nv
import matplotlib.pyplot as pp
import urllib.request
from MDAnalysis.analysis import distances
from IPython.display import Markdown, HTML
%matplotlib notebook

# PDB Ids
Retrieve PDBs (done already, no need to do again unless changed)

In [ ]:
pdb_ids = '1mey 4qf3 6uei 6uej 2puy 6fi1 6fhq 5yc3'.split()
pdb_ids += '3t7l 3u9g 4q6f 3iuf 4bbq 5yc4 6sxw 5y20'.split()
if False: #True to download again
    for pdb_id in pdb_ids:
        urllib.request.urlretrieve(
            f"https://files.rcsb.org/download/{pdb_id}.pdb",
            f"trajectories/pdbs/{pdb_id}.pdb")

### Total no. of PDBs studied

In [ ]:
len(pdb_ids)

In [ ]:
unis = {}#{'1znf' : mda.Universe("input.pdb")}
for pdb_id in pdb_ids:
    unis[pdb_id] = mda.Universe(f"trajectories/pdbs/{pdb_id}.pdb")

In [ ]:
def nearby_residues(uni, r=3):
    zns = uni.select_atoms('name ZN')
    
    shells = {}
    
    for ind in zns.indices:
        shell = uni.select_atoms(f'same residue as (around {r} index {ind})')
        #residues = [f'{atom.resname}{atom.resid}' for atom in shell]
        
        shells[ind] = shell.residues
        
    return shells

# Find pdb ids with list of Zincs that match 1ZNF

By "match" here I mean that there are 2 HIS and 2 CYS within given radius

In [ ]:
matching_zns = {}
matches_found = 0
for pid, uni in unis.items():
    sel = uni.select_atoms('name ZN')
    display(Markdown(f"### {pid} : {sel.n_atoms} Zn sites "))
    for ind, shell in nearby_residues(uni, r=3).items():
        his_count = np.count_nonzero(shell.resnames == 'HIS')
        cys_count = np.count_nonzero(shell.resnames == 'CYS')
        
        counts_match = his_count + cys_count == 4 
        #counts_match = his_count==2 and cys_count==2
        
        col = 'green' if counts_match else 'red'
        display(
            Markdown(f'  <span style="color:{col}"> {ind:5d} : {"--".join(shell.resnames)} </span>' )
        )
        if counts_match:
            matches_found += 1
            if pid not in matching_zns:
                matching_zns[pid] = []
            matching_zns[pid].append(ind)

In [ ]:
matches_found

# Get minimum Zn-water distance for each matching binding site

In [ ]:
smallest_d = 9
for pid, inds in matching_zns.items():
    display(Markdown(f"## {pid}"))
    for ind in inds:
        uni = unis[pid]
        zn = uni.select_atoms(f'index {ind}')
        waters = uni.select_atoms('resname HOH')
        dists = distances.distance_array(zn.positions, waters.positions)
        
        if len(waters.positions):
            mind = np.min(dists)
        else:
            print("No waters")
            continue
        
        if smallest_d > mind:
            smallest_d = mind
            
        if mind < 4: 
            col = 'red'
        else:
            col = 'white'
        display(Markdown(f' <span style="color:{col}"> {ind} : {mind:4.2f} </span>'))


In [ ]:
display(Markdown(f"**Smallest water distance found: {smallest_d:4.2f}**"))